In [1]:
import os
import re
from pytubefix import Playlist, YouTube
from tqdm import tqdm

def sanitize_filename(name):
    """Remove invalid filename characters"""
    return re.sub(r'[\\/*?:"<>|]', "", name)

def download_playlist(playlist_url, resolution="1080p", output_dir="playlists"):
    playlist = Playlist(playlist_url)
    playlist_title = sanitize_filename(playlist.title)
    output_path = os.path.join(output_dir, playlist_title)
    
    os.makedirs(output_path, exist_ok=True)
    
    for video in tqdm(playlist.videos, desc=f"Downloading '{playlist_title}'"):
        try:
            yt = YouTube(video.watch_url)
            base_filename = sanitize_filename(f"{yt.title}.mp4")
            final_path = os.path.join(output_path, base_filename)
            
            if os.path.exists(final_path):
                tqdm.write(f"Skipping existing: {base_filename}")
                continue
                
            # Select video stream
            video_stream = yt.streams.filter(res=resolution, mime_type="video/mp4").first()
            if not video_stream:
                video_stream = yt.streams.get_highest_resolution()
                
            # Select audio stream
            audio_stream = yt.streams.get_audio_only()
            
            # Download components
            video_path = video_stream.download(output_path=output_path, filename_prefix="video_")
            audio_path = audio_stream.download(output_path=output_path, filename_prefix="audio_")
            
            # Merge streams
            os.system(f"ffmpeg -y -i '{video_path}' -i '{audio_path}' -c:v copy -c:a aac '{final_path}' -loglevel error")
            
            # Cleanup temp files
            os.remove(video_path)
            os.remove(audio_path)
            
        except Exception as e:
            tqdm.write(f"Failed to download {video.title}: {str(e)}")

if __name__ == "__main__":
    #playlist_url = input("Enter YouTube Playlist URL: ")
    playlist_url = "https://youtube.com/playlist?list=PLvvHybQvq9efppZ31vapIksnYcSODIwFt&si=P50Hg-sUhGxTYUSy"
    download_playlist(playlist_url)


Failed to download Robotics and Automation Assignment/Project: [Errno 2] No such file or directory: '/Users/hochiyau/dev/youtube/playlists/Robotics and Automation Tutorial Solutions/video_Robotics and Automation Assignment/Project.mp4'
